## 数据规整：连接、联合、重塑

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows=20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure',figsize=(10,6))
np.set_printoptions(precision=4,suppress=True)

### 分层索引

In [4]:
data=pd.Series(np.random.randn(9),
              index=[['a','a','a','b','b','c','c','d','d'],
                     [1,2,3,1,3,1,2,2,3]])
data


a  1    1.669025
   2   -0.438570
   3   -0.539741
b  1    0.476985
   3    3.248944
c  1   -1.021228
   2   -0.577087
d  2    0.124121
   3    0.302614
dtype: float64

In [8]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [6]:
data['a']

1    1.669025
2   -0.438570
3   -0.539741
dtype: float64

In [10]:
data.unstack()
#stack 堆
## 好用

,1,2,3
a,1.669025,-0.438570,-0.539741
b,0.476985,NaN,3.248944
c,-1.021228,-0.577087,NaN
d,NaN,0.124121,0.302614


In [11]:
data.unstack(fill_value=0)

,1,2,3
a,1.669025,-0.438570,-0.539741
b,0.476985,0.000000,3.248944
c,-1.021228,-0.577087,0.000000
d,0.000000,0.124121,0.302614


In [15]:
frame=pd.DataFrame(np.arange(12).reshape((4,3)),
                  index=[['a','a','b','b'],
                         [1,2,1,2]],
                  columns=[['Ohio','Ohio','Colorado'],
                           ['Green','Red','Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

### 给横轴、纵轴两层索引分别加入表头

In [17]:
frame.index.names=['key1','key2']
frame.columns.names=['state:','color:']
frame

state:     Ohio     Colorado
color:    Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [20]:
frame.sort_index(level=1)

state:     Ohio     Colorado
color:    Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

### 交换层级,并按交换后的0层级排序


In [25]:
frame.swaplevel().sort_index(level=0)
# 此时level=0 是key_2
#     level=1 是key_1

state:     Ohio     Colorado
color:    Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 按层级进行汇总统计

In [26]:
frame.sum(level='key2')

C:\Users\DELL\AppData\Local\Temp/ipykernel_2260/2004046222.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')


state:  Ohio     Colorado
color: Green Red    Green
key2                     
1          6   8       10
2         12  14       16

In [31]:
frame.sum(level='color:',axis=1)

C:\Users\DELL\AppData\Local\Temp/ipykernel_2260/2736190728.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='color:',axis=1)


color:     Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## 使用dataframe的列进行索引

In [51]:
frame=pd.DataFrame({
    'a':range(7),
    'b':np.arange(7,0,-1),
    'c':['one','one','one','two','two','two','two'],
    'd':[0,1,2,0,1,2,3]    
})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [52]:
frame.set_index(['a','b'])

,,c,d
a,b,,
0,7,one,0
1,6,one,1
2,5,one,2
3,4,two,0
4,3,two,1
5,2,two,2
6,1,two,3


## 联合与合并数据集

In [54]:
df1=pd.DataFrame({
    'key':['b','b','a','c','a','a','b'],
    'data1':range(7)
})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [56]:
df2=pd.DataFrame({
    'key':['a','b','d'],
    'data2':range(3)
})
df2

,key,data2
0,a,0
1,b,1
2,d,2


### merge 归并

In [59]:
pd.merge(df1,df2)
# df1中的c 和 df2中的d舍去

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
